In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..')))

In [2]:
import ee
import geemap
from vegetation_period_NDVI.time_series import extract_time_ranges, get_harmonic_ts
from vegetation_period_extraction import (
    get_crop_veg_period,
    create_binary_ndvi_indicator,
    create_binary_mask,
)
from typing import List

In [3]:
ee.Initialize(project="thurgau-irrigation")

## Exporting the vegetation period images as an asset to the project

### Define the Area of Interest

In [4]:
cantonal_borders_asset = "projects/thurgau-irrigation/assets/Zuerich/Zuerich_bound"

aoi_feature_collection = ee.FeatureCollection(cantonal_borders_asset)
aoi_geometry = aoi_feature_collection.geometry()
aoi_geometry = aoi_geometry.simplify(500)
aoi_buffered = aoi_geometry.buffer(100)

aoi_name = "Zuerich"

### Define the years of interest

In [5]:
years = [2022]

### Export the Vegetation Periods images to the project assets

In [6]:
for year in years:

    print(f"Processing year {year}")

    time_intervals = extract_time_ranges([f"{year}-03-01", f"{year}-10-31"], 15)

    image_to_export = get_crop_veg_period(year, aoi_buffered, time_intervals)

    task_name = f"Zuerich_crop_veg_period_{year}"
    asset_id = f"projects/thurgau-irrigation/assets/Zuerich/crop_vegetation_period_zh_{year}"

    bands_to_export = [
        "firstStart",
        "firstEnd",
        "secondStart",
        "secondEnd",
        "isDoubleCropping",
    ]

    image_to_export = image_to_export.select(bands_to_export)

    task = ee.batch.Export.image.toAsset(
        image=image_to_export,
        description=task_name,
        assetId=asset_id,
        region=aoi_buffered,
        scale=10,
        maxPixels=1e13,
    )

    task.start()

    print(f"Exporting {task_name} to {asset_id}")

Processing year 2022
Exporting Zuerich_crop_veg_period_2022 to projects/thurgau-irrigation/assets/Zuerich/crop_vegetation_period_zh_2022


In [8]:
print(image_to_export.projection().getInfo())

{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}


In [ ]:
# veg_2019 = ee.Image("projects/thurgau-irrigation/assets/Thurgau/VegetationPeriod/crop_veg_period_2018")

# Map = geemap.Map()
# Map.centerObject(aoi_geometry, 10)
# options = {
#     "bands": ["isDoubleCropping"],
#     "min": 0,
#     "max": 1,
#     "palette": ["blue", "red"],
# }
# Map.addLayer(veg_2019, options, "Crop Veg Period 2019")

# Map

## General Code Testing. Ignore this part

In [ ]:
print(f"Processing year 2018")

time_intervals = extract_time_ranges([f"2018-03-01", f"2018-10-31"], 15)

image_to_export = get_crop_veg_period(2018, aoi_buffered, time_intervals)

In [ ]:
Map = geemap.Map()
Map.centerObject(aoi_geometry, 10)
options = {
    "bands": ["isDoubleCropping"],
    "min": 0,
    "max": 1,
    "palette": ["blue", "red"],
}
Map.addLayer(image_to_export, options, "Crop Veg Period 2019")

Map